# Daten von Songs über API abrufen

### REST API-Nutzung: Last.fm API
- Die Daten stammen von der öffentlichen Last.fm-API.
- Für den Zugriff wurde ein API-Key verwendet (Client-Zugriff ohne OAuth).
- Die Daten wurden über zwei Endpunkte abgefragt:
  - `chart.gettoptracks` für populäre Songs
  - `track.search` für einzelne Track-Details
- Die Antwort wurde als JSON empfangen und per `.json()` in Python verarbeitet.
- Die Ergebnisse wurden in ein CSV exportiert.


In [4]:
!pip install requests pandas

In [7]:
import requests
import pandas as pd

import requests

API_KEY = "767b0ae3f48164448f5ef3f525c631ed"
url = f"https://ws.audioscrobbler.com/2.0/?method=chart.gettoptracks&api_key={API_KEY}&format=json&limit=100"

response = requests.get(url)
data = response.json()


songs = [track['name'] for track in data['tracks']['track']]

print("Top 10 Songs:\n", songs[:10])


track_data = []


for song in songs:
    url = f"https://ws.audioscrobbler.com/2.0/?method=track.search&track={song}&api_key={API_KEY}&format=json&limit=1"
    response = requests.get(url)
    data = response.json()

    try:
        track = data["results"]["trackmatches"]["track"][0]
        track_data.append({
            "Track": track["name"],
            "Artist": track["artist"],
            "Listeners": track["listeners"],
            "URL": track["url"]
        })
    except (IndexError, KeyError):
        print(f"Kein Ergebnis für: {song}")


df = pd.DataFrame(track_data)
df.to_csv("lastfm_tracks.csv", index=False)


print(df)
print(f"\n{len(df)} Songs gespeichert in 'lastfm_tracks.csv'")


Top 10 Songs:
 ['Good Luck, Babe!', 'BIRDS OF A FEATHER', 'Party 4 U', 'See You Again (feat. Kali Uchis)', 'back to friends', 'No One Noticed', 'Manchild', 'espresso', 'undressed', 'Creep']
                                       Track              Artist Listeners  \
0                           Good Luck, Babe!       Chappell Roan   1683677   
1                         BIRDS OF A FEATHER       Billie Eilish   1785566   
2                                  Party 4 U          Charli XCX    767993   
3           See You Again (feat. Kali Uchis)  Tyler, the Creator   2112467   
4                            back to friends               Sombr    556017   
..                                       ...                 ...       ...   
95  Why'd You Only Call Me When You're High?      Arctic Monkeys   2284158   
96                        Fake Plastic Trees           Radiohead   1970963   
97                 Cigarettes out the Window             TV Girl   1285098   
98                           F